In [ ]:
!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow_text
!pip install datasets



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2"

In [ ]:
bert_preprocess = hub.KerasLayer(preprocess_url)

In [ ]:
dataset = load_dataset("mediabiasgroup/mbib-base", "cognitive-bias")

df= pd.DataFrame(dataset['train'])
#df = df.sample(frac = 0.03)
X = df['text']
y= df['label']

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
text_preprocessed = bert_preprocess(df["text"])
text_preprocessed.keys()

dict_keys(['input_mask', 'input_type_ids', 'input_word_ids'])

In [ ]:
text_preprocessed['input_mask']

<tf.Tensor: shape=(7092, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [ ]:
bert_encoder = hub.KerasLayer(encoder_url)

In [ ]:
bert_results = bert_encoder(text_preprocessed)

In [ ]:
bert_results.keys()

dict_keys(['default', 'encoder_outputs', 'sequence_output', 'pooled_output'])

In [ ]:
bert_results["pooled_output"] #embedding for the entire sentence

<tf.Tensor: shape=(7092, 128), dtype=float32, numpy=
array([[-0.99999094,  0.07336804, -0.99135154, ...,  0.9901099 ,
         0.94520897,  0.9676085 ],
       [-0.9998347 ,  0.05010236, -0.9888227 , ...,  0.9565068 ,
         0.6817847 ,  0.9894745 ],
       [-0.99852353, -0.02659259, -0.9960243 , ...,  0.9580887 ,
         0.9047884 ,  0.99519724],
       ...,
       [-0.99995387,  0.05352484, -0.98523635, ...,  0.99106574,
         0.9890929 ,  0.9942125 ],
       [-0.9999387 ,  0.04133975, -0.9982727 , ...,  0.9842626 ,
         0.934554  ,  0.9986218 ],
       [-0.99990785,  0.0197706 , -0.99936306, ...,  0.9693825 ,
         0.9686121 ,  0.99351   ]], dtype=float32)>

In [ ]:
bert_results["sequence_output"] #individual words embedding vectors. It is the
#same as the last encoder output


<tf.Tensor: shape=(7092, 128, 128), dtype=float32, numpy=
array([[[-0.7220757 , -0.57216483, -3.8679955 , ..., -0.9257057 ,
         -1.7967091 ,  1.4893506 ],
        [-0.9134617 ,  0.05592583, -0.48386675, ..., -1.995433  ,
          0.03582139,  0.65686464],
        [-1.185576  , -0.22558446,  0.06697592, ..., -2.4648998 ,
         -0.8990861 ,  2.6298795 ],
        ...,
        [-1.0149283 , -0.32500753, -0.8444824 , ..., -1.7225648 ,
         -0.7404627 ,  2.293339  ],
        [-0.75149417, -0.33793658, -0.75256544, ..., -1.4114588 ,
         -0.8547215 ,  2.020059  ],
        [-0.7379788 , -0.00701248, -0.66029155, ..., -1.4275272 ,
         -1.0628192 ,  2.0083609 ]],

       [[ 0.10377847,  0.45274982, -3.5517254 , ...,  0.8841338 ,
         -1.6002306 ,  1.221247  ],
        [-1.593793  ,  0.66634625,  0.1569472 , ..., -1.65858   ,
         -1.5236907 , -0.43111205],
        [-2.2061672 ,  0.45787776,  0.5569897 , ..., -1.0326381 ,
         -1.8691124 ,  0.09334254],
        .

In [ ]:
len(bert_results["encoder_outputs"]) #number of layers. Output of indicual encoder

2

In [ ]:
df.groupby("label").describe()

text                                                               
      count unique                                                top freq
label                                                                     
0      3546   3546  To vote in Mexico every eligible Mexican citiz...    1
1      3546   3546     His true name is Barak Hussein Muhammed Obama     1

In [ ]:
df['label'].value_counts()

0    3546
1    3546
Name: label, dtype: int64

In [ ]:
5129/(5041+5129)

0.5043264503441495

In [ ]:
df_hate = df[df['label']== 1]
df_hate.shape

(3546, 2)

In [ ]:
df_nice = df[df['label']== 0]
df_nice.shape

(3546, 2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df['label'])

In [ ]:
X_df = pd.DataFrame(X_train)
X_df.T.head(1).T

,text
2238,We got the Quonset Business Park for free and ...
855,Before I was state treasurer my Rhode Island b...
4804,Earlier this year John DePetro was accused of ...
620,Says David Rivera received a 5 000 campaign co...
3783,Says President Barack Obama did not invite a R...
...,...
65,Compact fluorescent light bulbs are toxic and ...
4107,Says Obama called Sarah Palin a pig
2035,The incoming Republican freshman class well it...
488,Budget analysts say the looming tax hikes and ...


In [ ]:
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder (text_preprocessed)['pooled_output']

#get_sentence_embeding(["We didn t buy at the perfect time "])



In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
 
text_preprocessed = bert_preprocess(text_input)

outputs = bert_encoder(text_preprocessed)
pooled_output = outputs['pooled_output']
#Neural Network Layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)


In [ ]:
#construct final model

model = tf.keras.Model(inputs=text_input, outputs=output)

In [ ]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_10 (KerasLayer)    {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [ ]:
METRICS= [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
167/167 [==============================] - 33s 179ms/step - loss: 0.7353 - accuracy: 0.5086 - precision: 0.5096 - recall: 0.4568
Epoch 2/50
167/167 [==============================] - 31s 184ms/step - loss: 0.7040 - accuracy: 0.5377 - precision: 0.5386 - recall: 0.5278
Epoch 3/50
167/167 [==============================] - 33s 198ms/step - loss: 0.6960 - accuracy: 0.5477 - precision: 0.5484 - recall: 0.5414
Epoch 4/50
167/167 [==============================] - 29s 174ms/step - loss: 0.6969 - accuracy: 0.5413 - precision: 0.5426 - recall: 0.5271
Epoch 5/50
167/167 [==============================] - 29s 175ms/step - loss: 0.6933 - accuracy: 0.5437 - precision: 0.5434 - recall: 0.5481
Epoch 6/50
167/167 [==============================] - 30s 178ms/step - loss: 0.6943 - accuracy: 0.5439 - precision: 0.5422 - recall: 0.5650
Epoch 7/50
167/167 [==============================] - 31s 186ms/step - loss: 0.6900 - accuracy: 0.5448 - precision: 0.5452 - recall: 0.5414
Epoch 8/50
167/167 [

In [ ]:
model.evaluate(X_test, y_test)

56/56 [==============================] - 11s 178ms/step - loss: 0.6744 - accuracy: 0.5809 - precision: 0.5795 - recall: 0.5880


[0.67439204454422, 0.5809362530708313, 0.57953280210495, 0.5880361199378967]